In [1]:
import json
from io import open

file_path = "2020_acl_diplomacy/data/train.jsonl"
train_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        train_data[i] = json.loads(line)
        i+=1




In [2]:
# print(len(diplomacy_data))
# for key, value in diplomacy_data[27].items():
#     print("Key: ", key)
messages_26= train_data[36]["messages"]
sender_26= train_data[36]["sender_labels"]
receiever_26= train_data[36]["receiver_labels"]

# print(len(messages_26), len(sender_26), len(receiever_26))

i=0
for message in messages_26:
    print(message, sender_26[i], receiever_26[i])
    i+=1


Hello Russia. Have you worked out what you’re doing with turkey and Austria yet? I’m thinking of going anti Germany but want to see how we align in the north first. False NOANNOTATION
I sent Turkey a message but still waiting on a reply. I wanted to go anti-Germany too if you want to split the population centers? But if I don’t hear back from Turkey I may have to fortify my South flank and enter the Balkan’s even though I don’t want to. True NOANNOTATION
I moving to pressureGermany but I could use assistance on your end. Willing to share any gains that could result. True True
Yikes. You have some Turkish trouble. I’ll plan on moving into helgoland bight True NOANNOTATION
Yea, that’s always annoying playing as Russia. Trying to get AH to help out but I have to move one of my troops back West away from pressuring Germany. Oh well. True True
That’s a shame. Let’s reassess after you handle your problems in the east. True NOANNOTATION
Hey man. Let me know if you wanna support my fleet movin

In [3]:
%pip install transformers
from transformers import T5Tokenizer


inputs, targets = [], []

for key, instance in train_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        inputs.append(f"context: {context} classify: {message}")
        targets.append("true" if label else "false")


print(len(inputs), len(targets)) 
print(inputs[3])
print(targets[3])

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13132 13132
context: Germany!

Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. 

So...do I suggest it?

I’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.

What is your appetite like for unusual and crazy? You've whet my appetite, Italy. What's the suggestion? 👍 classify: It seems like there are a lot of ways that could go wrong...I don't see why France would see you approaching/taking Munich--while I do nothing about it--and not immediately feel skittish
true


In [4]:
%pip install sentencepiece
tokenizer = T5Tokenizer.from_pretrained('t5-small')

train_encodings = tokenizer(inputs, truncation=True, padding='max_length', max_length=512)
train_labels = tokenizer(targets, truncation=True, padding='max_length', max_length=512)

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
%pip install torch
import torch
from torch.utils.data import Dataset, DataLoader

class DiplomacyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DiplomacyDataset(train_encodings, train_labels)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [9]:

file_path = "2020_acl_diplomacy/data/validation.jsonl"
validation_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        validation_data[i] = json.loads(line)
        i+=1




In [10]:
val_texts = []
targetval_texts = []

for key, instance in validation_data.items():
    messages_str = ' '.join(instance['messages'])
    labels_str = ' '.join(map(str, instance['sender_labels']))
    val_texts.append(messages_str)
    targetval_texts.append(labels_str)

In [13]:
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=512)
val_labels = tokenizer(targetval_texts, truncation=True, padding='max_length', max_length=512)
val_dataset = DiplomacyDataset(val_encodings, val_labels)


In [31]:
# %pip install allennlp
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
# from allennlp.training.metrics import F1Measure
# from allennlp.training.metrics.fbeta_measure import FBetaMeasure
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=1000,
    save_total_limit=3
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

for i in range(10):
    trainer.train()
    trainer.save_model(f"./results/model_{i}")
    results = trainer.evaluate(val_dataset)
    print(results)

    prediction_outputs = trainer.predict(val_dataset)
    predictions, true_labels = prediction_outputs.predictions, prediction_outputs.label_ids
    predicted_labels = np.argmax(prediction_outputs.predictions, axis=1)
    print(predicted_labels)
    # f1 = f1_score(true_labels, predictions)
    # print(f"F1 Score: {f1}")

    acc = accuracy_score(true_labels, predicted_labels)
    print(f"Accuracy: {acc}")

# print accuracy and loss at each step
# what is the input and what is expected output for both classification and generation




100%|██████████| 3/3 [00:21<00:00,  7.28s/it]


{'train_runtime': 21.8171, 'train_samples_per_second': 0.275, 'train_steps_per_second': 0.138, 'train_loss': 16.02013905843099, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<00:00, 45.71it/s]


{'eval_loss': 16.306488037109375, 'eval_runtime': 0.4979, 'eval_samples_per_second': 4.017, 'eval_steps_per_second': 2.009, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<00:00, 1444.82it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (2, 2, 512) + inhomogeneous part.